### Orbis and PPP Loan Data Merge

1. Extract matches between the PPP Loan Data and Orbis
2. Get all businesses NOT in Orbis but is in Bay Area AND is Minority-owned
3. Get needed columns from PPP Loan to append to Orbis
4. Extract PPP Loan columns and merge with orbis
5. Add PPP bus not in Orbis with those PPP loan columns
6. Reexport and update everyone

In [8]:
from cleanco import cleanco
import pandas as pd
import re
import os
from os.path import dirname, abspath
from IPython.display import display, HTML # Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

In [9]:
# Get Current Working Directory and Parent Path (for reading files in different folders)
## Source: https://stackoverflow.com/questions/30218802/get-parent-of-current-directory-from-python-script/30218825
d = dirname(dirname(abspath(os.getcwd())))
d

'C:\\Users\\Trang\\Desktop\\cp-research\\afn'

In [10]:
orbis_file = d+'/ppp-loan-data/Orbis database_six cities only.csv'
ppp_file =d+'/ppp-loan-data/out/bay_bus_from_ppp.csv'

In [11]:
orbis_df = pd.read_csv(orbis_file)
orbis_df.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Latitude,Longitude,Website address,"NAICS 2017, core code (4 digits)","NAICS 2017, core code - description",Size classification,Type of entity,Ethnic minority owned indicator (in US),Woman owned indicator (in US),Study Area
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412,LF,2019.0,...,"37° 48' 30.9"" N","122° 16' 1.2"" W",www.calnursesfoundation.org,8139,"Business, Professional, Labor, Political, and ...",Very large company,Corporate,No,No,Oakland
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519,LF,2019.0,...,"37° 49' 18.2"" N","122° 15' 37.2"" W",www.hondacarsofcorona.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519,LF,2019.0,...,"37° 49' 0.2"" N","122° 15' 48.6"" W",NaN,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778,LF,2019.0,...,"37° 46' 7.3"" N","122° 13' 10.9"" W",www.economylumberco.com,4441,Building Material and Supplies Dealers,Large company,Corporate,No,No,Oakland
4,BROADWAY FORD,No,No,No,No,No,US,4519,LF,2008.0,...,"37° 48' 51.1"" N","122° 15' 53.5"" W",www.mybroadwayford.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland


In [12]:
ppp_df = pd.read_csv(ppp_file)
ppp_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,NonProfit,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4
0,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,0,...,NaN,15281.29,05/13/2021,95125,Unanswered,2020,811121.0,"Automotive Body, Paint, and Interior Repair an...",Automotive Repair and Maintenance,8111.0
1,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,0,...,NaN,0.00,NaN,94510,No,2021,722330.0,Mobile Food Services,Special Food Services,7223.0
2,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,0,...,NaN,0.00,NaN,94107,No,2021,541511.0,Custom Computer Programming Services,Computer Systems Design and Related Services,5415.0
3,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,0,...,NaN,0.00,NaN,94551,Unanswered,2021,541420.0,Industrial Design Services,Specialized Design Services,5414.0
4,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,0,...,NaN,0.00,NaN,95113,Unanswered,2021,541110.0,Offices of Lawyers,Legal Services,5411.0


In [13]:
# Clean Orbis Name Column
orbis_df['Company Name - Clean'] =orbis_df['Company name Latin alphabet'].apply(lambda x: cleanco(x).clean_name())
orbis_df.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Longitude,Website address,"NAICS 2017, core code (4 digits)","NAICS 2017, core code - description",Size classification,Type of entity,Ethnic minority owned indicator (in US),Woman owned indicator (in US),Study Area,Company Name - Clean
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412,LF,2019.0,...,"122° 16' 1.2"" W",www.calnursesfoundation.org,8139,"Business, Professional, Labor, Political, and ...",Very large company,Corporate,No,No,Oakland,CALIFORNIA NURSES ASSOCIATION
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519,LF,2019.0,...,"122° 15' 37.2"" W",www.hondacarsofcorona.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland,HONDA CARS OF CORONA
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519,LF,2019.0,...,"122° 15' 48.6"" W",NaN,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland,SIMI MANAGEMENT
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778,LF,2019.0,...,"122° 13' 10.9"" W",www.economylumberco.com,4441,Building Material and Supplies Dealers,Large company,Corporate,No,No,Oakland,ECONOMY LUMBER CO OF OAKLAND
4,BROADWAY FORD,No,No,No,No,No,US,4519,LF,2008.0,...,"122° 15' 53.5"" W",www.mybroadwayford.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland,BROADWAY FORD


In [14]:
ppp_df['BorrowerName - Clean'] = ppp_df['BorrowerName'].apply(str).apply(lambda x: cleanco(x).clean_name())
ppp_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4,BorrowerName - Clean
0,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,0,...,15281.29,05/13/2021,95125,Unanswered,2020,811121.0,"Automotive Body, Paint, and Interior Repair an...",Automotive Repair and Maintenance,8111.0,MARKET AUTO TRUCK COLLISION
1,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,0,...,0.00,NaN,94510,No,2021,722330.0,Mobile Food Services,Special Food Services,7223.0,GOLD RUSH KETTLE KORN
2,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,0,...,0.00,NaN,94107,No,2021,541511.0,Custom Computer Programming Services,Computer Systems Design and Related Services,5415.0,KALIBER LABS
3,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,0,...,0.00,NaN,94551,Unanswered,2021,541420.0,Industrial Design Services,Specialized Design Services,5414.0,ROSS MCDONALD COMPANY
4,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,0,...,0.00,NaN,95113,Unanswered,2021,541110.0,Offices of Lawyers,Legal Services,5411.0,GATES EISENHART DAWSON


In [ ]:
# Find businesses in Orbis databases
ppp_from_orbis = ppp_df[ppp_df['BorrowerName - Clean'].isin(orbis_df['Company Name - Clean'])]
ppp_from_orbis.head()

In [ ]:
def extract_columns(df, columns):
    return df[columns].reset_index()